## Gaussian Process Classifier Using Tensorflow and Tensorflow-probability

In [6]:
#import tensorflow.compat.v1 as tf
#tf.compat.v1.enable_eager_execution()
import tensorflow.compat.v2 as tf
import tensorflow_probability as tfp
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
#tf.disable_v2_behavior()
import pandas as pd
import numpy as np
import os
import itertools
import sys
from sklearn.metrics import f1_score
import time

tfb = tfp.bijectors
tfd = tfp.distributions
tfk = tfp.math.psd_kernels

#tf.enable_v2_behavior()
#tf.executing_eagerly()

In [7]:
data = pd.read_csv('sonar.csv', header=None)
data = data.values
X = data[:,0:60].astype(float)
Y = data[:,60]

encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

X_train, X_test, y_train, y_test = train_test_split(X, encoded_Y, test_size=0.2, random_state=42)

In [8]:
# Defining kernel: RBF kernel with same length scale

constrain_positive = tfb.Shift(np.finfo(np.float64).tiny)(tfb.Softplus())

amplitude = tfp.util.TransformedVariable(
    initial_value=1.,
    bijector=constrain_positive,
    name='amplitude',
    dtype=np.float64)

length_scale = tfp.util.TransformedVariable(
    initial_value=1.,
    bijector=constrain_positive,
    name='length_scale',
    dtype=np.float64)

kernel = tfk.ExponentiatedQuadratic(amplitude, length_scale)

In [9]:
from TFBinaryGPClassifier import TFBinaryGPClassifier

start = time.time()

BGPC = TFBinaryGPClassifier(kernel=kernel).fit(X=X_train, y=y_train)
probs_preds = BGPC.predict_proba(X=X_test)
labels_preds = BGPC.predict(X=X_test)

end =time.time()

In [10]:
print('Accuracy and F1 accuracy are %f %f' %(np.mean(labels_preds==y_test),f1_score(labels_preds, y_test)), 'in %f seconds' %(end-start))
print('Optmized amplitude and length scale are %f, %f' %(BGPC.kernel.amplitude, BGPC.kernel.length_scale))

Accuracy and F1 accuracy are 0.857143 0.823529 in 2.271332 seconds
Optmized amplitude and length scale are 2.874847, 0.775601
